(C) 2025 Zachary Gonsalves Based on UK Government data published by the UK Health Security Agency and on the DIY Disease Tracking Dashboard Kit by Fabrizio Smeraldi. Released under the GNU GPLv3.0 or later.

# Antimicrobial Resistance in Streptococcus pneumoniae Dashboard

This dashboard displays trends in antimicrobial resistance tested in Streptococcus pneumoniae (S-pneumoniae) over a multi-year period.  

In [4]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import json
from ipywidgets import interact

class APIwrapper:
    _access_point="https://api.ukhsa-dashboard.data.gov.uk"
    _last_access=0.0 # time of last api access
   
    def __init__(self, theme, sub_theme, topic, geography_type, geography, metric):
        """ Init the APIwrapper object, constructing the endpoint from the structure
        parameters """
        url_path=(f"/themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/" +
                  f"{geography_type}/geographies/{geography}/metrics/{metric}")
        self._start_url=APIwrapper._access_point+url_path
        self._filters=None
        self._page_size=-1
        self.count=None

    def get_page(self, filters={}, page_size=5):
        """ Access the API and download the next page of data. Sets the count
        attribute to the total number of items available for this query. Changing
        filters or page_size will cause get_page to restart from page 1. Rate
        limited to three request per second. The page_size parameter sets the number
        of data points in one response page (maximum 365); use the default value 
        for debugging your structure and filters. """
        # Check page size is within range
        if page_size>365:
            raise ValueError("Max supported page size is 365")
        # restart from first page if page or filters have changed
        if filters!=self._filters or page_size!=self._page_size:
            self._filters=filters
            self._page_size=page_size
            self._next_url=self._start_url
        # signal the end of data condition
        if self._next_url==None: 
            return [] # we already fetched the last page
        # simple rate limiting to avoid bans
        curr_time=time.time() # Unix time: number of seconds since the Epoch
        deltat=curr_time-APIwrapper._last_access
        if deltat<0.33: # max 3 requests/second
            time.sleep(0.33-deltat)
        APIwrapper._last_access=curr_time
        # build parameter dictionary by removing all the None
        # values from filters and adding page_size
        parameters={x: y for x, y in filters.items() if y!=None}
        parameters['page_size']=page_size
        # the page parameter is already included in _next_url.
        # This is the API access. Response is a dictionary with various keys.
        # the .json() method decodes the response into Python object (dictionaries,
        # lists; 'null' values are translated as None).
        response = requests.get(self._next_url, params=parameters).json()
        # update url so we'll fetch the next page
        self._next_url=response['next']
        self.count=response['count']
        # data are in the nested 'results' list
        return response['results'] 

    def get_all_pages(self, filters={}, page_size=365):
        """ Access the API and download all available data pages of data. Sets the count
        attribute to the total number of items available for this query. API access rate
        limited to three request per second. The page_size parameter sets the number
        of data points in one response page (maximum 365), and controls the trade-off
        between time to load a page and number of pages; the default should work well 
        in most cases. The number of items returned should in any case be equal to 
        the count attribute. """
        data=[] # build up all data here
        while True:
            # use get_page to do the job, including the pacing
            next_page=self.get_page(filters, page_size)
            if next_page==[]:
                break # we are done
            data.extend(next_page)
        return data


#def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    #df=pd.DataFrame(index=range(0,100), columns=['One', 'Two'])
    #######################REDO


def api_button_callback(button):
    print("API refresh triggered at:", time.ctime()) # a test to see the function is being called properly
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    #global df
    #df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    refresh_timeseries_graph(apidata)
    refresh_drugtype_graph(data)
    # disable button after one use
    apibutton.icon="check"
    apibutton.disabled=True

def refresh_timeseries_graph(jdata):
    data={}
    for dataset in [data_numbertested, data_percentresistant, data_percentresistant]:
        for entry in dataset:
            date=entry['date']
            metric=entry['metric']
            value=entry['metric_value']
            if date not in data:
                data[date]={}
            data[date][metric]=value
   
    dates=list(data.keys())
    dates.sort()
    #find earliest and latest date and convert them to pandas date format by calling 'parse_date' function
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    #print (startdate, ' to ', enddate)

    index=pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['numbertested', 'percentresistant', 'percenttested'])
    timeseriesdf

    #Now we fill the data frame with values from our three statistics.
    metrics ={'numbertested': 's-pneumoniae_testing_bacteraemiaNumberTestedRollingMonth',
          'percentresistant': 's-pneumoniae_testing_bacteraemiaPercentResistantRollingMonth',
          'percenttested': 's-pneumoniae_testing_bacteraemiaPercentTestedRollingMonth'}

    for date, entry in data.items(): # each entry is a dictionary with each metric
        pd_date=parse_date(date) # convert to Pandas format
        for column in ['numbertested', 'percentresistant', 'percenttested']: 
            metric_name=metrics[column]
            # for dates with no values available, 0.0 is put
            value= entry.get(metric_name, 0.0)
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
            timeseriesdf.loc[date, column]=value
    # fill in any remaining "holes" due to missing dates
    timeseriesdf.fillna(0.0, inplace=True)
    timeseriesdf.to_pickle("timeseriesdf.pkl")
    graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})
    display(controls, graph)

def refresh_drugtype_graph(jdata):
    # Plotting the percentage resistant by drug type
    data={}
    for entry in data_percentresistant:
        date=entry['date']
        drugtype=entry['stratum']
        value=entry['metric_value']
        if date not in data:
            data[date]={}
        data[date][drugtype]=value

    dates=list(data.keys())
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    drugtype=[]
    for entry in data.values():
        for drg in entry.keys():
            if drg not in drugtype:
                drugtype.append(drg)
    drugtype.sort()

    index=pd.date_range(startdate, enddate, freq='W-MON')
    drugtypedf=pd.DataFrame(index=index, columns=drugtype)


    for date, entry in data.items(): # each entry is a dic with some drug types and values
        pd_date=parse_date(date) # convert to Pandas format
        for column in entry.keys(): # the drug types
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
            drugtypedf.loc[date, column]=entry[column]

    # fill in any remaining "holes" due to missing dates and variants
    drugtypedf.fillna(0.0, inplace=True)
    # adjust types of columns filled in
    drugtypedf.infer_objects(copy=False)

    # 1Q means "by quarter"; compute the average row
    quarterly= drugtypedf.groupby(pd.Grouper(freq='1QE')).mean() 
    totals=quarterly.sum(axis=1) # sum over the rows
    # make sure it's all normalised to 100
    quarterly=quarterly.div(totals, axis=0)*100 # divide the columns by the totals
    # reverse the rows for the graph, so older dates will be on top
    quarterly = quarterly[::-1]
    drugtypedf.to_pickle("drugtypedf.pkl")
    
    year=wdg.Select(
    # options available: unique years in the dataframe
    options=drugtypedf.index.year.unique(), # options available
    value=drugtypedf.index.year[-1], # initial value: most recent year
    rows=1, # rows of the selection box
    description='Year',
    disabled=False
    )
    # keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
    output=wdg.interactive_output(drugtype_graph, {'graphyear': year})

    display(year, output)


def access_api():
    structure={"theme": "infectious_disease",
           "sub_theme": "antimicrobial_resistance",
           "topic": "S-pneumoniae",
           "geography_type":"Nation",
           "geography": "England"}
    #get info for first metric
    structure["metric"]="s-pneumoniae_testing_bacteraemiaNumberTestedRollingMonth"
    api=APIwrapper(**structure)
    data=api.get_page()
    
    filters={"stratum" : None, 
         "age": None, 
         "sex": None,  
         "year": 2024,  
         "month": None, 
         "epiweek" :None, 
         "date" : None, 
         "in_reporting_delay_period": None 
          }
    numbertested=api.get_all_pages()
    with open("bacteraemiaNumberTested.json","wt") as OUTF:
        json.dump(numbertested, OUTF)
    
    #get info for second metric
    structure["metric"]="s-pneumoniae_testing_bacteraemiaPercentResistantRollingMonth"

    api=APIwrapper(**structure)
    data=api.get_page()
    filters={"stratum" : None, # Smallest subgroup a metric can be broken down into e.g. ethnicity, testing pillar
         "age": None, # Smallest subgroup a metric can be broken down into e.g. 15_44 for the age group of 15-44 years
         "sex": None, #  Patient gender e.g. 'm' for Male, 'f' for Female or 'all' for all genders
         "year": 2024, #  Epi year of the metrics value (important for annual metrics) e.g. 2020
         "month": None, # Epi month of the metric value (important for monthly metrics) e.g. 12
         "epiweek" :None, # Epi week of the metric value (important for weekly metrics) e.g. 30
         "date" : None, # The date which this metric value was recorded in the format YYYY-MM-DD e.g. 2020-07-20
         "in_reporting_delay_period": None # Boolean indicating whether the data point is considered to be subject to retrospective updates
        }
    percentresistant=api.get_all_pages()
    #store collected info for second metric in json file
    with open ("percentresistant.json","wt") as OUTF:
        json.dump(percentresistant, OUTF)
    #get info for third metric
    structure["metric"]="s-pneumoniae_testing_bacteraemiaPercentTestedRollingMonth"
    api=APIwrapper(**structure)
    data=api.get_page() # default size is 5
    filters={"stratum" : None, 
         "age": None, 
         "sex": None,  
         "year": 2024,  
         "month": None, 
         "epiweek" :None, 
         "date" : None, 
         "in_reporting_delay_period": None 
          }
    percenttested=api.get_all_pages()
    #store data for third metric in json file
    with open ("percenttested.json","wt") as OUTF:
        json.dump(percenttested, OUTF)
     
    #loading initial data from disk
    with open("bacteraemiaNumberTested.json") as file1:
        data_numbertested=json.load(file1)
    with open ("percentresistant.json") as file2:
        data_percentresistant=json.load(file2)
    with open ("percenttested.json") as file3:
        data_percenttested=json.load(file3)

    #storing loaded data in variable
    jsondata={ "data_numbertested": data_numbertested,
         "data_percentresistant": data_percentresistant,
         "data_percenttested": data_percenttested
    }
         
    apibutton.icon="check"
    apibutton.disabled=True
    return jsondata

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        ax=timeseriesdf[list(gcols)].plot(logy=logscale)
        ax.set_title('S-Pneumoniae Testing Bacteraemia Rolling Month') 
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

def drugtype_graph(graphyear):
    yeardf=drugtypedf[drugtypedf.index.year==graphyear]
    monthly= yeardf.groupby(pd.Grouper(freq='1ME')).mean() 
    totals=monthly.sum(axis=1) # over the rows
    # normalise to 100
    monthly=monthly.div(totals, axis=0)*100
    # older dates on top of the graph
    monthly = monthly[::-1]
    ax=monthly.plot(kind='barh', stacked=True,cmap='tab20')
    ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
    ax.set_yticklabels(monthly.index.strftime('%Y-%m-%d'))
    ax.set_title('Percentage S-Pneumoniae resistance to Macrolides vs Penicillin');
    plt.show() # important - graphs won't update if this is missing 

def timeseries_graph2(graphyear):
    yeardf=timeseriesdf[timeseriesdf.index.year==graphyear]
    monthly= yeardf.groupby(pd.Grouper(freq='1ME')).mean() 
    totals=monthly.sum(axis=1) # over the rows
    # normalise to 100
    monthly=monthly.div(totals, axis=0)*100
    # older dates on top of the graph
    monthly = monthly[::-1]
    ax=monthly.plot(kind='barh', stacked=True,cmap='tab20')
    ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
    ax.set_yticklabels(monthly.index.strftime('%Y-%m-%d'))
    ax.set_title('S-Pneumoniae Testing Bacteraemia Rolling Month') 
    plt.show() # important - graphs won't update if this is missing 


# This dashboard will concentrate on the sub-theme of antimicrobial resistance and the topic of Streptococcus pneumoniae (S-pneumoniae) 
# There are three metrics for this topic, 's-pneumoniae_testing_bacteraemiaNumberTestedRollingMonth,' 's-pneumoniae_testing_bacteraemiaPercentResistantRollingMonth'
# and 's-pneumoniae_testing_bacteraemiaPercentTestedRollingMonth.'
#structure={"theme": "infectious_disease",
#           "sub_theme": "antimicrobial_resistance",
#           "topic": "S-pneumoniae",
#           "geography_type":"Nation",
#           "geography": "England"}
        

#get info for first metric
#structure["metric"]="s-pneumoniae_testing_bacteraemiaNumberTestedRollingMonth"

#api=APIwrapper(**structure)
#data=api.get_page() # default size is 5
#if api.count==0:
#    print("Data not available in this selection")
#else:print(api.count)
#print(data)

#filters={"stratum" : None, 
#         "age": None, 
#         "sex": None,  
#         "year": 2024,  
#         "month": None, 
#         "epiweek" :None, 
#         "date" : None, 
#         "in_reporting_delay_period": None 
#        }

#test to see data is being pulled successfully
#data_2024=api.get_page(filters,page_size=5)
#print(api.count)
#print(data_2024)
#numbertested=api.get_all_pages()
#print(f"Data points expected: {api.count}")
#print(f"Data points retrieved: {len(numbertested)}")

#store the collected data in a json file
#with open("bacteraemiaNumberTested.json","wt") as OUTF:
#    json.dump(numbertested, OUTF)

#get info for second metric
#structure["metric"]="s-pneumoniae_testing_bacteraemiaPercentResistantRollingMonth"

#api=APIwrapper(**structure)
#data=api.get_page() # default size is 5
#if api.count==0:
#    print("Data not available in this selection")
#else:print(api.count)
#print(data)

#filters={"stratum" : None, # Smallest subgroup a metric can be broken down into e.g. ethnicity, testing pillar
         #"age": None, # Smallest subgroup a metric can be broken down into e.g. 15_44 for the age group of 15-44 years
         #"sex": None, #  Patient gender e.g. 'm' for Male, 'f' for Female or 'all' for all genders
         #"year": 2024, #  Epi year of the metrics value (important for annual metrics) e.g. 2020
         #"month": None, # Epi month of the metric value (important for monthly metrics) e.g. 12
         #"epiweek" :None, # Epi week of the metric value (important for weekly metrics) e.g. 30
         #"date" : None, # The date which this metric value was recorded in the format YYYY-MM-DD e.g. 2020-07-20
         #"in_reporting_delay_period": None # Boolean indicating whether the data point is considered to be subject to retrospective updates
         #}

#data_2024=api.get_page(filters,page_size=5)
#print(api.count)
#print(data_2024)
#percentresistant=api.get_all_pages()
#print(f"Data points expected: {api.count}")
#print(f"Data points retrieved: {len(percentresistant)}")

#store collected info for second metric in json file
#with open ("percentresistant.json","wt") as OUTF:
#    json.dump(percentresistant, OUTF)

#get info for third metric
#structure["metric"]="s-pneumoniae_testing_bacteraemiaPercentTestedRollingMonth"

#api=APIwrapper(**structure)
#data=api.get_page() # default size is 5
#if api.count==0:
#    print("Data not available in this selection")
#else:print(api.count)
#print(data)

#filters={"stratum" : None, 
#         "age": None, 
#         "sex": None,  
#         "year": 2024,  
#         "month": None, 
#         "epiweek" :None, 
#         "date" : None, 
#         "in_reporting_delay_period": None 
#        }

#test to see data is being pulled successfully
#data_2024=api.get_page(filters,page_size=5)
#print(api.count)
#print(data_2024)
#percenttested=api.get_all_pages()
#print(f"Data points expected: {api.count}")
#print(f"Data points retrieved: {len(percenttested)}")

#store data for third metric in json file
#with open ("percenttested.json","wt") as OUTF:
#    json.dump(percenttested, OUTF)

#Data is collected, now we can move onto processing it 

#loading initial data from disk
with open("bacteraemiaNumberTested.json") as file1:
    data_numbertested=json.load(file1)

with open ("percentresistant.json") as file2:
    data_percentresistant=json.load(file2)

with open ("percenttested.json") as file3:
    data_percenttested=json.load(file3)

#storing loaded data in variable
jsondata={ "data_numbertested": data_numbertested,
          "data_percentresistant": data_percentresistant,
          "data_percenttested": data_percenttested
         }

#df=wrangle_data(jsondata)

# iPython enabling of the embedding of matplotlib output
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

#data_percenttested[:5] #just to show how the data is stored, and how we must go about retrieving the values to plot the graphs

#retrieving the 'date' and 'metric' values by looping over the dataset and storing them in a dictionary with the date as the key and three metrics as values
data={}
for dataset in [data_numbertested, data_percentresistant, data_percentresistant]:
    for entry in dataset:
        date=entry['date']
        metric=entry['metric']
        value=entry['metric_value']
        if date not in data:
            data[date]={}
        data[date][metric]=value

#data
#data['2023-02-01']

#extracting all dates and sorting them
dates=list(data.keys())
dates.sort()
#dates

#find earliest and latest date and convert them to pandas date format by calling 'parse_date' function
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])
#print (startdate, ' to ', enddate)

#Defining data frame. Index is created as 'date range' , date analog of a range for integers, will include dates missing from the list. Data frame is defined,
#specifying index and title of its columns 
index=pd.date_range(startdate, enddate, freq='D')
timeseriesdf=pd.DataFrame(index=index, columns=['numbertested', 'percentresistant', 'percenttested'])
timeseriesdf

#Now we fill the data frame with values from our three statistics. I have chosen to use the same methodology to do so as shown in the lecture. 
metrics ={'numbertested': 's-pneumoniae_testing_bacteraemiaNumberTestedRollingMonth',
          'percentresistant': 's-pneumoniae_testing_bacteraemiaPercentResistantRollingMonth',
          'percenttested': 's-pneumoniae_testing_bacteraemiaPercentTestedRollingMonth'}

for date, entry in data.items(): # each entry is a dictionary with each metric
    pd_date=parse_date(date) # convert to Pandas format
    for column in ['numbertested', 'percentresistant', 'percenttested']: 
        metric_name=metrics[column]
        # for dates with no values available, 0.0 is put
        value= entry.get(metric_name, 0.0)
        # this is the way you access a specific location in the dataframe - use .loc
        # and put index,column in a single set of [ ]
        timeseriesdf.loc[date, column]=value
            
# fill in any remaining "holes" due to missing dates
timeseriesdf.fillna(0.0, inplace=True)
            
#timeseriesdf

# At this point we begin plotting

#ax=timeseriesdf.plot() 
#ax.set_title('S-pneumoniae antimicrobial resistance tested: Number Tested Rolling Month, Percent Resistant Rolling Month, Percent Tested Rolling Month');
#ax=timeseriesdf.plot(logy=True)
#ax.set_title('S-pneumoniae antimicrobial resistance tested: Number Tested Rolling Month, Percent Resistant Rolling Month, Percent Tested Rolling Month');


# Plotting the percentage resistant by drug type
data={}
for entry in data_percentresistant:
    date=entry['date']
    drugtype=entry['stratum']
    value=entry['metric_value']
    if date not in data:
        data[date]={}
    data[date][drugtype]=value

#display the munged data

#data

dates=list(data.keys())
dates.sort()
#dates

startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])
#print (startdate, ' to ', enddate)

drugtype=[]
for entry in data.values():
    for drg in entry.keys():
        if drg not in drugtype:
            drugtype.append(drg)
drugtype.sort()
#print(drugtype)

index=pd.date_range(startdate, enddate, freq='W-MON')
drugtypedf=pd.DataFrame(index=index, columns=drugtype)
#drugtypedf

for date, entry in data.items(): # each entry is a dic with some drug types and values
    pd_date=parse_date(date) # convert to Pandas format
    for column in entry.keys(): # the drug types
        # this is the way you access a specific location in the dataframe - use .loc
        # and put index,column in a single set of [ ]
        drugtypedf.loc[date, column]=entry[column]

# fill in any remaining holes due to missing dates and variants
drugtypedf.fillna(0.0, inplace=True)
# adjust types of columns filled in
drugtypedf.infer_objects(copy=False)
            
#drugtypedf

# 1Q - by quarter, compute the average row
quarterly= drugtypedf.groupby(pd.Grouper(freq='1QE')).mean() 
totals=quarterly.sum(axis=1) # sum over the rows
# make sure it's all normalised to 100
quarterly=quarterly.div(totals, axis=0)*100 # divide the columns by the totals
# reverse the rows for the graph, so older dates will be on top
quarterly = quarterly[::-1]

#colormap
#ax=quarterly.plot(kind='barh', stacked=True,cmap='tab20')
#ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
# date formatting
#ax.set_yticklabels(quarterly.index.strftime('%Y-%m-%d'))
#ax.set_title('Percentage S-Pneumoniae resistance to Macrolides and Penicillin');

#save wrangled data to pickle files
timeseriesdf.to_pickle("timeseriesdf.pkl")
drugtypedf.to_pickle("drugtypedf.pkl")


#Now adding interactive controls

#Read the data from the pickle files
timeseriesdf=pd.read_pickle("timeseriesdf.pkl")
drugtypedf=pd.read_pickle("drugtypedf.pkl")

#The first button,  a dropdown selection menu
series=wdg.SelectMultiple(
    options=['numbertested', 'percentresistant'],
    value=['numbertested', 'percentresistant'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([series, scale])

graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})
display(controls, graph)

year=wdg.Select(
    # options available: unique years in the dataframe
    options=drugtypedf.index.year.unique(), # options available
    value=drugtypedf.index.year[-1], # initial value: most recent year
    rows=1, # rows of the selection box
    description='Year',
    disabled=False
)

# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(drugtype_graph, {'graphyear': year})

display(year, output)


apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='refresh' 
)

# On hitting refresh, the button will go through the api_button_callback function to 
apibutton.on_click(api_button_callback)
display(apibutton)

/tmp/ipykernel_168/3426068240.py:512: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  timeseriesdf.fillna(0.0, inplace=True)
/tmp/ipykernel_168/3426068240.py:566: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  drugtypedf.fillna(0.0, inplace=True)


Output()

Select(description='Year', index=5, options=(2020, 2021, 2022, 2023, 2024, 2025), rows=1, value=2025)

Output()

Button(description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click to download current Pub…

None

Graph 1: Rolling Month Resistance tested: Displays the number of samples tested and the percentage of samples demonstrating antibiotic resistance. Select both options on the log scale for a clearer comparison of the percent resistances against the total sample sizes. 

Graph 2: Comparison of resistance demonstrated against 2 antibiotics; macrolides and penicillin. Clear indication that s-pneumoniae exhibits far more resistance on average to macrolides than penicillin. 


Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

This is a template for your DIY Disease Tracking Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

**Author and License** Remember that if you deploy your dashboard as a Binder it will be publicly accessible. Change the copyright notice and take credit for your work! Also acknowledge your sources and the conditions of the license by including this notice: "Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/)."